In [1]:
import pandas as pd

from autogluon.tabular import TabularPredictor
from autogluon.core.metrics import make_scorer
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve, auc

In [2]:
def pr_auc_Metric(y_true, y_pred):
    precision, recall, _ = precision_recall_curve(y_true, y_pred)
    score = auc(recall, precision)
    return score

In [3]:
ag_roc_auc_scorer = make_scorer(
    name='roc_auc',
    score_func=pr_auc_Metric,
    optimum=1,
    greater_is_better=True,
    needs_proba=True
)

In [4]:
TIME_LIMIT = 2 * 3600

In [5]:
extra_metrics = ['roc_auc', 'accuracy', 'balanced_accuracy', 'f1', 'f1_macro', 'f1_micro', 'average_precision', 'precision', 'recall', 'log_loss', 'pac_score']

In [6]:
for i in range(0, 10):
    print(f'Fitting model {i}')
    original_data = pd.read_csv(
        f'../../data/train/set_{i}.csv'
        )
    synthetic_data = pd.read_csv(
        f'../../data/synthetic/ctgan/set_{i}.csv'
    )
    # Conteo de clases
    original_vc = original_data['ED_2Clases'].value_counts()
    synthetic_vc = synthetic_data['ED_2Clases'].value_counts()
    print(f"Original data: {original_vc['D']} {original_vc['H']}")
    print(f"Synthetic data: {synthetic_vc['D']} {synthetic_vc['H']}")

    # Calcular cuántos 'D' sintéticos tomar
    d_sample = original_vc['H'] + synthetic_vc['H'] - original_vc['D']
    print(f"Sampled data: {d_sample}")

    # Seleccionar datos sintéticos
    synth_sampled = pd.concat([
        synthetic_data[synthetic_data['ED_2Clases'] == 'H'],
        synthetic_data[synthetic_data['ED_2Clases'] == 'D'].sample(d_sample, random_state=42)
    ])

    # Combinar y mezclar
    train_data = pd.concat([original_data, synth_sampled], ignore_index=True).sample(frac=1, random_state=42).reset_index(drop=True)
    print(f"Train data: {train_data['ED_2Clases'].value_counts()}")

    predictor= TabularPredictor(
        label='ED_2Clases',
        problem_type='binary',
        eval_metric=ag_roc_auc_scorer,
        sample_weight='balance_weight',
        path=f'AutogluonModels/mix/v{i}'
    )

    predictor.fit(
        train_data = train_data,
        time_limit = TIME_LIMIT,
        presets = ['high_quality'],
        auto_stack = False,
        excluded_model_types=['KNN', 'LR'],
        num_stack_levels=0,
        num_bag_folds=0,
        verbosity = 2
    )

    df_test = pd.read_csv(
        f'../../data/test/set_{i}.csv'
    )

    predictors = predictor.leaderboard(
        df_test,
        extra_metrics = extra_metrics,
        extra_info=True,
        silent=True
    )
    predictors.to_excel(
        f'../../5_results/5_3_mix/predictors_set_{i}.xlsx',
        index=False
    )

    feature_importance = predictor.feature_importance(train_data)
    feature_importance.to_excel(
        f'../../5_results/5_3_mix/feature_importance_set_{i}.xlsx',
        index=True
    )

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.11.11
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
CPU Count:          12
Memory Avail:       7.45 GB / 15.94 GB (46.8%)
Disk Space Avail:   71.30 GB / 446.36 GB (16.0%)
Presets specified: ['high_quality']


Fitting model 0
Original data: 223 59
Synthetic data: 59 223
Sampled data: 59
Train data: ED_2Clases
D    282
H    282
Name: count, dtype: int64


Using predefined sample weighting strategy: balance_weight. Evaluation metrics will ignore sample weights, specify weight_evaluation=True to instead report weighted metrics.
Beginning AutoGluon training ... Time limit = 7200s
AutoGluon will save models to "c:\Users\jgala\uned\tfm\synthetic-data\4_ml\4_3_mix\AutogluonModels\mix\v0"
Train Data Rows:    564
Train Data Columns: 19
Label Column:       ED_2Clases
Problem Type:       binary
Preprocessing data ...
Selected class <--> label mapping:  class 1 = H, class 0 = D
	Note: For your binary classification, AutoGluon arbitrarily selected which label-value represents positive (H) vs negative (D) class.
	To explicitly set the positive_class, either rename classes to 1 and 0, or specify positive_class in Predictor init.
Assigning sample weights to balance differences in frequency of classes.
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    7623.46 MB
	Train Da

[1000]	valid_set's binary_logloss: 0.431734


	0.92s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: RandomForest_r39 ... Training model for up to 7149.96s of the 7149.95s of remaining time.
	0.865	 = Validation score   (roc_auc)
	0.66s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: CatBoost_r167 ... Training model for up to 7149.24s of the 7149.23s of remaining time.
	0.8879	 = Validation score   (roc_auc)
	1.45s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: NeuralNetFastAI_r95 ... Training model for up to 7147.77s of the 7147.76s of remaining time.
No improvement since epoch 3: early stopping
	0.727	 = Validation score   (roc_auc)
	0.82s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: NeuralNetTorch_r41 ... Training model for up to 7146.94s of the 7146.93s of remaining time.
	0.9064	 = Validation score   (roc_auc)
	1.33s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: XGBoost_r98 ... Training model for up to 7145.59s of the 7145.58s of rema

...

	0.9061	 = Validation score   (roc_auc)
	2.16s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: CatBoost_r128 ... Training model for up to 7114.82s of the 7114.81s of remaining time.
	0.8686	 = Validation score   (roc_auc)
	4.78s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: NeuralNetFastAI_r111 ... Training model for up to 7110.04s of the 7110.03s of remaining time.
	0.89	 = Validation score   (roc_auc)
	2.17s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: NeuralNetTorch_r31 ... Training model for up to 7107.84s of the 7107.83s of remaining time.
	0.9122	 = Validation score   (roc_auc)
	1.53s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: ExtraTrees_r4 ... Training model for up to 7106.29s of the 7106.28s of remaining time.
	0.8559	 = Validation score   (roc_auc)
	0.55s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: NeuralNetFastAI_r65 ... Training model for up to 7105.68s of the 7105.68s of rem

[1000]	valid_set's binary_logloss: 0.364407


	0.9032	 = Validation score   (roc_auc)
	1.26s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: XGBoost_r49 ... Training model for up to 7099.28s of the 7099.28s of remaining time.


[2000]	valid_set's binary_logloss: 0.355482


	0.8949	 = Validation score   (roc_auc)
	0.59s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: CatBoost_r5 ... Training model for up to 7098.68s of the 7098.67s of remaining time.
	0.8753	 = Validation score   (roc_auc)
	1.01s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: NeuralNetTorch_r87 ... Training model for up to 7097.65s of the 7097.64s of remaining time.
	0.8992	 = Validation score   (roc_auc)
	3.01s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: NeuralNetTorch_r71 ... Training model for up to 7094.61s of the 7094.61s of remaining time.
	0.9027	 = Validation score   (roc_auc)
	0.48s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: CatBoost_r143 ... Training model for up to 7094.12s of the 7094.11s of remaining time.
	0.892	 = Validation score   (roc_auc)
	1.88s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: ExtraTrees_r178 ... Training model for up to 7092.24s of the 7092.23s of remaining t